In [ ]:
import os
import numpy as np
import json
import pandas as pd
# tasks = [
#     "agieval", "boolq", "hellaswag", "mathqa", "mrpc", "piqa", "qnli", "rte", "social_iqa", "winogrande",
#     "arc_challenge", "copa", "lambada_standard",  "multirc", "pubmedqa", "qqp", "sciq", "sst2", "wnli",
#     "arc_easy", "gsm8k", "logiqa", "mnli", "openbookqa", "qasper", "race", "scrolls", "wikitext", "wsc"
# ]
tasks = {'group1': ['wikitext', 'lambada_standard'],
'group2': ['triviaqa', 'nq_open', 'webqs', 'hellaswag', 'squadv2'],
'group3': ['squadv2']}
tasks['all'] = tasks['group1'] + tasks['group2'] + tasks['group3']
#
tasks = tasks['group1']
# tasks = ['sst2']

In [ ]:
step_to_versions = {('1b', 'iter-600000-ckpt-step-75000_hf'):  ['cc_merged_v2_8k', 'cc_merged_v1_8k', 'intramask_cc_8k', 'intramask_cc_merged_v2_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v2_8k_intracccont','cc_8k',],
                    ('1b', 'iter-480000-ckpt-step-60000_hf'):  ['cc_merged_v2_8k', 'intramask_cc_8k', 'intramask_cc_merged_v2_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v2_8k_intracccont','cc_8k',],
                    ('1b', 'iter-380000-ckpt-step-47500_hf'):  ['cc_merged_v2_8k', 'intramask_cc_8k', 'intramask_cc_merged_v2_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v2_8k_intracccont','cc_8k',],
                    }

In [ ]:
all_dfs = []
nshot = 10
size='1b'
# iter_name = 'iter-160000-ckpt-step-40000_hf'
# iter_name = 'iter-480000-ckpt-step-60000_hf'
iter_name = 'iter-600000-ckpt-step-75000_hf'
# iter_name = 'iter-380000-ckpt-step-47500_hf'
for task_name in tasks:
    task_performance = {}
    # for ds_version in ['cc', 'cc_merged_v1', 'cc_merged_v2', 'cc_merged_v3', 'intramask_cc']:
    # for ds_version in ['cc_merged_v2_8k', 'intramask_cc_8k', 'intramask_cc_merged_v2_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v2_8k_intrav2cont','cc_8k',]:
    # for ds_version in ['cc_merged_v2_8k', 'cc_merged_v1_8k', 'intramask_cc_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v2_8k_intracccont','cc_8k',]: #
    # intramask_cc_merged_v2_8k
    # for ds_version in ['baseline']:
    for ds_version in ['BM25Chunk', "UniChunk", "MixChunk", "IntraDoc"]:

        # BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_{ds_version}_8k/iter-380000-ckpt-step-47500_hf"
        # BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/home/aiops/zhuty/tinyllama/out/tiny_LLaMA_{size}_8k_{ds_version}/{iter_name}"
        BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/tyzhu/tiny_LLaMA_{size}_8k_{ds_version}_{iter_name}"
        BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/yuzhaouoe/{ds_version}"
    #
    #     BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/TinyLlama/TinyLlama-1.1B-step-50K-105b"
        # BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/home/aiops/zhuty/lm_indexer_data/tyzhu/flan_max_300_added_tyzhu_tiny_LLaMA_1b_8k_{ds_version}_8k_iter-380000-ckpt-step-47500_hf/checkpoint-12387"
#         result_file_path = os.path.join(BASE_PATH, f"{task_name}", f"results_{nshot}.json")
        # result_file_path = os.path.join(BASE_PATH, f"{task_name}", f"{nshot}", f"tyzhu__tiny_LLaMA_{size}_8k_{ds_version}_{iter_name}")
        # result is a file that begins with 'result', get the latest result
        # result_files = [f for f in os.listdir(result_file_path) if f.startswith('result')]
        # result_files.sort()
        # result_file_path = os.path.join(result_file_path, result_files[-1])
        result_file_path = os.path.join(BASE_PATH, f"{task_name}", f"{nshot}", f"results.json")
        if not os.path.exists(result_file_path):
            print(f"File not found", task_name, ds_version)
            continue
        result = json.load(open(result_file_path))
        # print(result)
        assert result['n-shot'][task_name] == nshot, "n-shot not match, got %d" % result['n-shot'][task_name] + " expected %d" % nshot + "in task %s" % task_name + "of model %s" % ds_version
        result = result['results'][task_name]
        task_performance[ds_version] = result
        # break
    # make the dataframe
    print("Task:", task_name)
    if len(task_performance) == 0:
        continue
    df = pd.DataFrame(task_performance)
    df = pd.concat([df.loc[['alias']], df.drop('alias')])
    all_dfs.append(df)
    print(df)

In [ ]:
new_df = pd.concat(all_dfs, axis=0)

In [ ]:
new_df

#### Compare the same model over different steps

In [ ]:
all_dfs = []
nshot = 0
size='360M'
for task_name in tasks:
    task_performance = {}
    # for ds_version in ['cc_merged_v2_8k', 'intramask_cc_8k', 'intramask_cc_merged_v2_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v2_8k_intrav2cont','cc_8k',]:
    ds_version = 'cc_merged_v2_8k_intrav2cont'
    for step in range(27500, 67500 + 2500, 2500):
    # for step in range(17500, 80000 + 2500, 2500):
        # BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_{ds_version}_8k/iter-380000-ckpt-step-47500_hf"
        BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/home/aiops/zhuty/tinyllama/out/tiny_LLaMA_{size}_8k_{ds_version}/iter-{step*4:06}-ckpt-step-{step}_hf"
        iter_name = f"iter-{step*4:06}-ckpt-step-{step}_hf"

        # BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/home/aiops/zhuty/lm_indexer_data/tyzhu/flan_max_300_added_tyzhu_tiny_LLaMA_1b_8k_{ds_version}_8k_iter-380000-ckpt-step-47500_hf/checkpoint-12387"
        # result_file_path = os.path.join(BASE_PATH, f"{task_name}", f"results_{nshot}.json")
        # result_file_path = os.path.join(BASE_PATH, f"{task_name}", f"{nshot}", f"results.json")
        result_file_path = os.path.join(BASE_PATH, f"{task_name}", f"{nshot}", f"__home__aiops__zhuty__tinyllama__out__tiny_LLaMA_{size}_8k_{ds_version}__{iter_name}")
        # result is a file that begins with 'result', get the latest result
        result_files = [f for f in os.listdir(result_file_path) if f.startswith('result')]
        result_files.sort()
        result_file_path = os.path.join(result_file_path, result_files[-1])


        if not os.path.exists(result_file_path):
            print(f"File not found", task_name, result_file_path)
            continue
        result = json.load(open(result_file_path))
        # print(result)
        assert result['n-shot'][task_name] == nshot, "n-shot not match, got %d" % result['n-shot'][task_name] + " expected %d" % nshot + "in task %s" % task_name + "of model %s" % ds_version
        result = result['results'][task_name]
        task_performance[ds_version] = result
        # break
    # make the dataframe
        print("Task:", task_name)
        if len(task_performance) == 0:
            continue
        df = pd.DataFrame(task_performance)
        df = pd.concat([df.loc[['alias']], df.drop('alias')])
        df = df.transpose()
        df['step'] = step
        all_dfs.append(df)

In [ ]:
concat_df = pd.concat(all_dfs, axis=0)

In [ ]:
concat_df

In [ ]:
# plot the word perplexity against the step
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(concat_df['step'], concat_df['word_perplexity,none'], label=f"{task_name}-{ds_version}")
plt.xlabel('step')
plt.ylabel('word_perplexity')
plt.show()


### Testing the tokenizer

In [ ]:
from transformers import LlamaTokenizer
import json

In [ ]:
# model = LlamaForConditionalGeneration.from_pretrained("tyzhu/tiny_LLaMA_1b_8k_cc_merged_v3_8k")
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", token = json.load(open('/home/aiops/zhuty/hf_token.json')))

In [ ]:
tokenizer.decode([13])

In [ ]:
tokenizer.encode("How am I doing? \n\n "
                 "I am doing well")

In [ ]:
commands = []
# for step in range(17500, 80000+2500, 2500):
for step in range(27500, 67500 + 2500, 2500):
    # commands.append(f" bash convert_to_hf_general.sh /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_360M_8k_cc_merged_v2_8k_intrav2cont  iter-{step*4:06}-ckpt-step-{step}")
    commands.append(f" bash eval.sh  wikitext /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_360M_8k_cc_merged_v2_8k_intrav2cont/iter-{step*4:06}-ckpt-step-{step}_hf 0")

In [ ]:
print(";".join(commands))

In [ ]:
from huggingface_hub import HfApi

# Initialize the API client
api = HfApi()
api_token = json.load(open("/home/aiops/zhuty/hf_token.json"))

# List models with the specified name
models = api.list_models(author="tyzhu", use_auth_token=api_token)
for model in models:
    model_id = model.modelId
    print(f"Making model '{model_id}' private")
    api.update_repo_visibility(repo_id=model_id, private=True, token=api_token)

In [ ]:
from huggingface_hub import HfApi
import json
for step in range(5000, 80000, 5000):
    try:
        HfApi().model_info(f"tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-{step*8:06}-ckpt-step-{step}_hf", token =json.load(open('/home/aiops/zhuty/hf_token.json')))
    except:
        print(f"tyzhu/tiny_LLaMA_1b_8k_cc_8k_iter-{step*8:06}-ckpt-step-{step}_hf" + " not found")